In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

#Moon equation form the Parisian Alfonsines Tables
Equation_center=HTable.read(181,format='dishas')
Equation_anomaly=HTable.read(182,format='dishas')
Div_diam=HTable.read(240,format='dishas')
Min_prop=HTable.read(239,format='dishas')
Hist_table=HTable.read(319)


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(15):
    for j in range(60):
        Test_recomp[i][1][j][1]=Sexagesimal('0;')*u.degree
        Delta_recomp[i][1][j][1]=Sexagesimal('0;')*u.degree


In [4]:


# moon mean motion parameter from the Parisian Alfonsine tables
daily_mean_center=Sexagesimal("13;10,35,1,15,11,4,35")*u.degree
daily_mean_arg=Sexagesimal("13;3,53,57,30,21,4,13")*u.degree
daily_mean_elong=Sexagesimal("12;11,26,41,37,51,50,39")*u.degree
REVO=Sexagesimal("6,0;0,0")*u.degree

for i in range(15):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):
        # initial value for mmoon age
        moon_age=i+1
        
        # Initial values for mean moon correction
        mean_moon_corr=moon_age*daily_mean_center
                        
        # Initial values for mean moon elongation
        mean_elong=2*moon_age*daily_mean_elong
        
        # computation of the equation of center and proportional minutes
        eq_center=Equation_center.get(mean_elong%REVO)
        min_prop1=Min_prop.get(mean_elong%REVO)

            
    for j in range(60):
        with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):

            # Initial values for mean argument
            mean_argument1= Hist_table[0][1][j][0]*u.degree
            mean_argument2= mean_argument1+daily_mean_arg*moon_age

            
            # Computation of the true argument
            true_argument=mean_argument2+eq_center   
           
            # computation of the first equation of anomaly
            eq_anomaly1=Equation_anomaly.get(true_argument%REVO)

            #computation of proportional minutes with the diversity of diameter
            div_diam=Div_diam.get(true_argument%REVO)
            min_prop2=min_prop1*div_diam/(60*u.degree)
                
            #correction of the equation of anomaly with the minuta proportionalia and diversity of diameter
            if eq_anomaly1>0:
                eq_anomaly2=eq_anomaly1+min_prop2
            else:
                eq_anomaly2=eq_anomaly1-min_prop2
    
    
        # computation of the final correction
        with set_precision(tmode=TruncatureMode.ROUND, pmode=1):
            moon_corr_fin=mean_moon_corr+eq_anomaly2
            moon_corr_fin=Historical(moon_corr_fin.value,1)         
            Test_recomp[i][1][j][1]=moon_corr_fin
            Delta_recomp[i][1][j][1]=60*(Hist_table[i][1][j][1]-Test_recomp[i][1][j][1])

In [5]:
# residuals visualisation and statistical summaries 1
A=Delta_recomp.to_pandas()
(A.mean().mean(), A.std().std())

(1.775555555555556, 164.71791965494967)

In [6]:
# residuals visualisation and statistical summaries 2
A.astype(float).style.background_gradient(axis=None)

,1 ;,2 ;,3 ;,4 ;,5 ;,6 ;,7 ;,8 ;,9 ;,10 ;,11 ;,12 ;,13 ;,14 ;,15 ;
06 ;,1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,2.000000
12 ;,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
18 ;,0.000000,-2.000000,0.000000,-1.000000,0.000000,-10.000000,1.000000,-479.000000,0.000000,0.000000,0.000000,0.000000,10.000000,1.000000,3.000000
24 ;,0.000000,120.000000,0.000000,-1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
01s 00 ;,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-1.000000,0.000000,1.000000,0.000000,1.000000,11.000000
01s 06 ;,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-3.000000,1.000000,0.000000,1.000000,-19.000000
01s 12 ;,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3600.000000,-1.000000,1.000000,2.000000,-6.000000,0.000000,1.000000,1.000000,2.000000
01s 18 ;,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,1.000000,0.000000,1.000000
01s 24 ;,0.000000,0.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
02s 00 ;,1.000000,0.000000,0.000000,2.000000,0.000000,-2.000000,0.000000,-1.000000,-1.000000,-1.000000,1.000000,-1.000000,-1.000000,0.000000,0.000000


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='True_moon_TM.csv')
B.to_csv('True_moon_TM.zip', index=False, compression=compression_opts)